In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

from tqdm import tqdm


2024-05-20 18:17:45.237809: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 18:17:46.561932: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 18:17:46.574280: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 18:17:51.834985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1.Create model

In [2]:
# Define the CNN model
def create_cnn_model(input_shape):
    model = models.Sequential()

    # Add Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    # Add Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))  # 2 output neurons for linear and angular speed

    return model

# Create an instance of the model
input_shape = (200, 200, 1)  # Define the input shape based on your camera image dimensions
model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Using Mean Squared Error as loss for regression

# Print model summary
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 135424)            0

# 2. Load data

In [13]:
import os
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import img_to_array

def load_img(path):
    try:
        path = os.path.join('/home/miquel/Escritorio/VAR/P2',str(path))
        image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (input_shape[0], input_shape[1]))
        image = image / 255.0
        image = img_to_array(image)
        image = image.reshape((input_shape[0], input_shape[1]))
        return image
    except:
        print(path)
        return None


paths = [f'data/dataset{i+1}.json' for i in range(4)]
dfs = []
for path in paths:
    dfs.append(pd.read_json(path))

df = pd.concat(dfs, axis=0)
df = df.reset_index()
images = []
labels = []
for i, path in tqdm(enumerate(df['path'])):
    images.append(load_img(path))
    labels.append(np.array([np.float32(df.at[i,'V']), np.float32(df.at[i,'W'])]))

# tqdm.pandas()
# images = np.array(df['path'].progress_apply(lambda x: load_img(x)).values)
# labels = np.array(df.progress_apply(
#     lambda x: 
#         np.array([np.float32(x['V']), np.float32(x['W'])]),
#     axis=1).values)

#train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

images = np.array(images)
labels = np.array(labels)
images, labels = shuffle(images, labels, random_state=42)

idx = int(len(images)*0.75)
train_images, train_labels = images[:idx], labels[:idx]
val_images, val_labels = images[idx:], labels[idx:]

print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"train_labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"val_images shape: {val_images.shape}, dtype: {val_images.dtype}")
print(f"val_labels shape: {val_labels.shape}, dtype: {val_labels.dtype}")


0it [00:00, ?it/s]

2195it [00:13, 221.70it/s][ WARN:0@17282.289] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_0.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.289] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.291] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.292] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.293] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.294] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_5.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.294] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_6.jpg'): can't open/read file: check file pa

/data/imgs_3/frame_0.jpg
/data/imgs_3/frame_1.jpg
/data/imgs_3/frame_2.jpg
/data/imgs_3/frame_3.jpg
/data/imgs_3/frame_4.jpg
/data/imgs_3/frame_5.jpg
/data/imgs_3/frame_6.jpg
/data/imgs_3/frame_7.jpg
/data/imgs_3/frame_8.jpg
/data/imgs_3/frame_9.jpg
/data/imgs_3/frame_10.jpg
/data/imgs_3/frame_11.jpg
/data/imgs_3/frame_12.jpg
/data/imgs_3/frame_13.jpg
/data/imgs_3/frame_14.jpg
/data/imgs_3/frame_15.jpg
/data/imgs_3/frame_16.jpg
/data/imgs_3/frame_17.jpg
/data/imgs_3/frame_18.jpg
/data/imgs_3/frame_19.jpg
/data/imgs_3/frame_20.jpg
/data/imgs_3/frame_21.jpg
/data/imgs_3/frame_22.jpg
/data/imgs_3/frame_23.jpg
/data/imgs_3/frame_24.jpg
/data/imgs_3/frame_25.jpg
/data/imgs_3/frame_26.jpg
/data/imgs_3/frame_27.jpg
/data/imgs_3/frame_28.jpg
/data/imgs_3/frame_29.jpg
/data/imgs_3/frame_30.jpg
/data/imgs_3/frame_31.jpg
/data/imgs_3/frame_32.jpg
/data/imgs_3/frame_33.jpg
/data/imgs_3/frame_34.jpg
/data/imgs_3/frame_35.jpg
/data/imgs_3/frame_36.jpg
/data/imgs_3/frame_37.jpg
/data/imgs_3/frame_38.

[ WARN:0@17282.449] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_345.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.449] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_346.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.449] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_347.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.449] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_348.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.450] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_349.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.450] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_350.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.450] global loadsave.cpp:248 findDecoder imread_('/data/imgs_3/frame_351.jpg'): can't open/read file: check file path/integrity

/data/imgs_3/frame_424.jpg
/data/imgs_3/frame_425.jpg
/data/imgs_3/frame_426.jpg
/data/imgs_3/frame_427.jpg
/data/imgs_3/frame_428.jpg
/data/imgs_3/frame_429.jpg
/data/imgs_3/frame_430.jpg
/data/imgs_3/frame_431.jpg
/data/imgs_3/frame_432.jpg
/data/imgs_3/frame_433.jpg
/data/imgs_3/frame_434.jpg
/data/imgs_3/frame_435.jpg
/data/imgs_3/frame_436.jpg
/data/imgs_3/frame_437.jpg
/data/imgs_3/frame_438.jpg
/data/imgs_3/frame_439.jpg
/data/imgs_3/frame_440.jpg
/data/imgs_3/frame_441.jpg
/data/imgs_3/frame_442.jpg
/data/imgs_3/frame_443.jpg
/data/imgs_3/frame_444.jpg
/data/imgs_3/frame_445.jpg
/data/imgs_3/frame_446.jpg
/data/imgs_3/frame_447.jpg
/data/imgs_3/frame_448.jpg
/data/imgs_3/frame_449.jpg
/data/imgs_3/frame_450.jpg
/data/imgs_3/frame_451.jpg
/data/imgs_3/frame_452.jpg
/data/imgs_3/frame_453.jpg
/data/imgs_3/frame_454.jpg
/data/imgs_3/frame_455.jpg
/data/imgs_3/frame_456.jpg
/data/imgs_3/frame_457.jpg
/data/imgs_3/frame_458.jpg
/data/imgs_3/frame_459.jpg
/data/imgs_3/frame_460.jpg
/

...

[ WARN:0@17282.865] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_328.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.865] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_329.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.865] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_330.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.866] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_331.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.866] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_332.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.866] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_333.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17282.866] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_334.jpg'): can't open/read file: check file path/integrity

/data/imgs_4/frame_358.jpg
/data/imgs_4/frame_359.jpg
/data/imgs_4/frame_360.jpg
/data/imgs_4/frame_361.jpg
/data/imgs_4/frame_362.jpg
/data/imgs_4/frame_363.jpg
/data/imgs_4/frame_364.jpg
/data/imgs_4/frame_365.jpg
/data/imgs_4/frame_366.jpg
/data/imgs_4/frame_367.jpg
/data/imgs_4/frame_368.jpg
/data/imgs_4/frame_369.jpg
/data/imgs_4/frame_370.jpg
/data/imgs_4/frame_371.jpg
/data/imgs_4/frame_372.jpg
/data/imgs_4/frame_373.jpg
/data/imgs_4/frame_374.jpg
/data/imgs_4/frame_375.jpg
/data/imgs_4/frame_376.jpg
/data/imgs_4/frame_377.jpg
/data/imgs_4/frame_378.jpg
/data/imgs_4/frame_379.jpg
/data/imgs_4/frame_380.jpg
/data/imgs_4/frame_381.jpg
/data/imgs_4/frame_382.jpg
/data/imgs_4/frame_383.jpg
/data/imgs_4/frame_384.jpg
/data/imgs_4/frame_385.jpg
/data/imgs_4/frame_386.jpg
/data/imgs_4/frame_387.jpg
/data/imgs_4/frame_388.jpg
/data/imgs_4/frame_389.jpg
/data/imgs_4/frame_390.jpg
/data/imgs_4/frame_391.jpg
/data/imgs_4/frame_392.jpg
/data/imgs_4/frame_393.jpg
/data/imgs_4/frame_394.jpg
/

[ WARN:0@17283.071] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_440.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17283.075] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_441.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17283.075] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_442.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17283.081] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_443.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17283.086] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_444.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17283.092] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_445.jpg'): can't open/read file: check file path/integrity
[ WARN:0@17283.092] global loadsave.cpp:248 findDecoder imread_('/data/imgs_4/frame_446.jpg'): can't open/read file: check file path/integrity

/data/imgs_4/frame_456.jpg
/data/imgs_4/frame_457.jpg
/data/imgs_4/frame_458.jpg
/data/imgs_4/frame_459.jpg
/data/imgs_4/frame_460.jpg
/data/imgs_4/frame_461.jpg
/data/imgs_4/frame_462.jpg
/data/imgs_4/frame_463.jpg
/data/imgs_4/frame_464.jpg
/data/imgs_4/frame_465.jpg
/data/imgs_4/frame_466.jpg
/data/imgs_4/frame_467.jpg
/data/imgs_4/frame_468.jpg
/data/imgs_4/frame_469.jpg
/data/imgs_4/frame_470.jpg
/data/imgs_4/frame_471.jpg
/data/imgs_4/frame_472.jpg
/data/imgs_4/frame_473.jpg
/data/imgs_4/frame_474.jpg
/data/imgs_4/frame_475.jpg
/data/imgs_4/frame_476.jpg
/data/imgs_4/frame_477.jpg
/data/imgs_4/frame_478.jpg
/data/imgs_4/frame_479.jpg
/data/imgs_4/frame_480.jpg
/data/imgs_4/frame_481.jpg
/data/imgs_4/frame_482.jpg
/data/imgs_4/frame_483.jpg
/data/imgs_4/frame_484.jpg
/data/imgs_4/frame_485.jpg
/data/imgs_4/frame_486.jpg
/data/imgs_4/frame_487.jpg
/data/imgs_4/frame_488.jpg
/data/imgs_4/frame_489.jpg
/data/imgs_4/frame_490.jpg
/data/imgs_4/frame_491.jpg
/data/imgs_4/frame_492.jpg
/

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3204,) + inhomogeneous part.

In [17]:
print(images.shape)
print(images[0].shape)
print(images[0][0].shape)
print(images[0][0][0].shape)
print(images[0][0][0])



for i, img in enumerate(images):
    print(img.shape, end='')
    if(i%13==0):
        print()


(2202,)
(200, 200)
(200,)
()
0.47843137
(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(2

# 3. Train

In [7]:

model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define callbacks (optional but useful for saving the best model)
checkpoint_path = "best_model.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model
batch_size = 32
epochs = 10

history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[checkpoint])

# Evaluate the model on the validation set
loss, mae = model.evaluate(val_images, val_labels)
print("Validation Loss:", loss)
print("Validation Mean Absolute Error:", mae)

# Load the best model
best_model = tf.keras.models.load_model(checkpoint_path)

# Make predictions on the validation set using the best model
predictions = best_model.predict(val_images)

# Optionally, you can save the best model for later use
best_model.save("data/models/best_model_trained.h5")

Epoch 1/10
49/49 [==============================] - ETA: 0s - loss: 0.6242 - mae: 0.4128
Epoch 1: val_loss improved from inf to 0.08426, saving model to best_model.h5


/home/miquel/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


49/49 [==============================] - 174s 3s/step - loss: 0.6242 - mae: 0.4128 - val_loss: 0.0843 - val_mae: 0.1599
Epoch 2/10
49/49 [==============================] - ETA: 0s - loss: 0.0606 - mae: 0.1444
Epoch 2: val_loss improved from 0.08426 to 0.02782, saving model to best_model.h5
49/49 [==============================] - 165s 3s/step - loss: 0.0606 - mae: 0.1444 - val_loss: 0.0278 - val_mae: 0.0798
Epoch 3/10
49/49 [==============================] - ETA: 0s - loss: 0.0286 - mae: 0.0958
Epoch 3: val_loss did not improve from 0.02782
49/49 [==============================] - 160s 3s/step - loss: 0.0286 - mae: 0.0958 - val_loss: 0.0325 - val_mae: 0.1285
Epoch 4/10
49/49 [==============================] - ETA: 0s - loss: 0.0227 - mae: 0.0882
Epoch 4: val_loss improved from 0.02782 to 0.02410, saving model to best_model.h5
49/49 [==============================] - 188s 4s/step - loss: 0.0227 - mae: 0.0882 - val_loss: 0.0241 - val_mae: 0.0887
Epoch 5/10
49/49 [========================